In [1]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

C:\Users\OW93790\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Read in DataFrame of stored images and types
pokedex_df=pd.read_csv("Data/Pokedex.csv").drop(labels="Unnamed: 0",axis=1)
# Convert "Type" column (read in as strings) back to lists
pokedex_df.Type=pokedex_df.Type.str[1:-1].str.split(', ').tolist()
pokedex_df.head()

,Nat_Dex,Name,Type,Files
0,1,Bulbasaur,"['Grass', 'Poison']",001Bulbasaur.png
1,2,Ivysaur,"['Grass', 'Poison']",002Ivysaur.png
2,3,Venusaur,"['Grass', 'Poison']",003Venusaur.png
3,4,Charmander,['Fire'],004Charmander.png
4,5,Charmeleon,['Fire'],005Charmeleon.png


In [3]:
# Generate the trainer from the images and types using Keras
# Validation split chosen so that the datagens are not prime numbers and may be run in reasonable batches
datagen=ImageDataGenerator(rescale=1./255,validation_split=204/809)

# Set shuffle to false for training set
train_generator=datagen.flow_from_dataframe(dataframe=pokedex_df, directory="Images", x_col="Files", y_col="Type", subset="training", class_mode="categorical", target_size=(250,250), seed=42, batch_size=6, shuffle=False)

valid_generator=datagen.flow_from_dataframe(dataframe=pokedex_df, directory="Images", x_col="Files", y_col="Type", subset="validation", class_mode="categorical", target_size=(250,250), seed = 42, batch_size=7, shuffle=True)

Found 606 images belonging to 18 classes.
Found 203 images belonging to 18 classes.


In [4]:
# Check logic of the classes
train_generator.classes
# Looks good. 18 classes--one for each type.
# Classes are keyed alphabetically, each pokemon has one or both of its types in original order

[[0],
 [0, 16],
 [12],
 [10, 7],
 [16, 10],
 [4],
 [4],
 [17, 13],
 [0, 16],
 [0, 15],
 [0, 5],
 [1, 11],
 [12],
 [12],
 [6],
 [6, 15],
 [11, 10],
 [11, 10],
 [17, 15],
 [17],
 [17],
 [11, 7],
 [17, 7],
 [16, 7],
 [1, 6],
 [1, 6],
 [17, 2],
 [10],
 [10],
 [12],
 [12],
 [12],
 [5],
 [5],
 [11, 14],
 [3],
 [6],
 [12],
 [12],
 [3],
 [6],
 [17],
 [15, 10],
 [15, 10],
 [15, 1],
 [14, 7],
 [6, 7],
 [14, 9],
 [9],
 [9],
 [9],
 [6],
 [6, 5],
 [6, 5],
 [17],
 [17, 10],
 [17, 10],
 [1],
 [1],
 [12],
 [12],
 [0],
 [0],
 [0, 7],
 [0],
 [0, 13],
 [17, 9],
 [17, 9],
 [17, 9],
 [9],
 [9, 1],
 [9, 1],
 [12, 7],
 [12, 7],
 [17, 7],
 [17, 7],
 [14, 4],
 [14, 4],
 [14, 4],
 [0, 17],
 [0, 7],
 [9],
 [9, 5],
 [12],
 [12],
 [12],
 [0, 10],
 [0, 7],
 [0, 8],
 [12],
 [12],
 [12],
 [5],
 [5],
 [12, 4],
 [15],
 [12],
 [12],
 [1, 8],
 [16, 4],
 [16, 15],
 [16, 15],
 [16, 15],
 [5, 14],
 [5, 14],
 [3],
 [3],
 [3],
 [3],
 [0],
 [0],
 [9, 13],
 [13],
 [13],
 [17, 1],
 [17, 1],
 [17],
 [17],
 [6, 10],
 [6, 10],
 [6]

In [5]:
# Model setting code from https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(250,250,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
# Reduced number of nodes to reduce overtraining
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(18, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [6]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5
)

Epoch 1/5
101/101 [==============================] - 308s 3s/step - loss: 4.4185 - acc: 0.0660 - val_loss: 4.1803 - val_acc: 0.1182
Epoch 2/5
101/101 [==============================] - 311s 3s/step - loss: 4.3650 - acc: 0.0908 - val_loss: 4.1574 - val_acc: 0.1626
Epoch 3/5
101/101 [==============================] - 316s 3s/step - loss: 4.3100 - acc: 0.1073 - val_loss: 4.1227 - val_acc: 0.1232
Epoch 4/5
101/101 [==============================] - 314s 3s/step - loss: 4.2596 - acc: 0.1188 - val_loss: 4.1161 - val_acc: 0.1084
Epoch 5/5
101/101 [==============================] - 311s 3s/step - loss: 4.1633 - acc: 0.1254 - val_loss: 4.0735 - val_acc: 0.1133


In [7]:
 # Save the model
model.save("Models/pkmn_model_best1.h5")

In [8]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5
)

Epoch 1/5
101/101 [==============================] - 315s 3s/step - loss: 4.0573 - acc: 0.1634 - val_loss: 4.0554 - val_acc: 0.1626
Epoch 2/5
101/101 [==============================] - 314s 3s/step - loss: 3.8445 - acc: 0.2013 - val_loss: 3.9392 - val_acc: 0.1724
Epoch 3/5
101/101 [==============================] - 312s 3s/step - loss: 3.6858 - acc: 0.2690 - val_loss: 4.0940 - val_acc: 0.1232
Epoch 4/5
101/101 [==============================] - 314s 3s/step - loss: 3.4667 - acc: 0.2970 - val_loss: 4.0183 - val_acc: 0.1182
Epoch 5/5
101/101 [==============================] - 312s 3s/step - loss: 3.2310 - acc: 0.3564 - val_loss: 3.9398 - val_acc: 0.1478


In [9]:
 # Save the model
model.save("Models/pkmn_model_best2.h5")

In [10]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5
)

Epoch 1/5
101/101 [==============================] - 310s 3s/step - loss: 3.1071 - acc: 0.3647 - val_loss: 4.1138 - val_acc: 0.1034
Epoch 2/5
101/101 [==============================] - 310s 3s/step - loss: 2.9427 - acc: 0.4076 - val_loss: 4.1967 - val_acc: 0.1281
Epoch 3/5
101/101 [==============================] - 315s 3s/step - loss: 2.7712 - acc: 0.4422 - val_loss: 4.2589 - val_acc: 0.1232
Epoch 4/5
101/101 [==============================] - 312s 3s/step - loss: 2.6679 - acc: 0.4703 - val_loss: 4.1037 - val_acc: 0.1478
Epoch 5/5
101/101 [==============================] - 314s 3s/step - loss: 2.6108 - acc: 0.4620 - val_loss: 4.3092 - val_acc: 0.1034


In [11]:
 # Save the model
model.save("Models/pkmn_model_best3.h5")

In [12]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5
)

Epoch 1/5
101/101 [==============================] - 315s 3s/step - loss: 2.5243 - acc: 0.4752 - val_loss: 4.3648 - val_acc: 0.1330
Epoch 2/5
101/101 [==============================] - 310s 3s/step - loss: 2.4224 - acc: 0.5215 - val_loss: 4.7692 - val_acc: 0.0837
Epoch 3/5
101/101 [==============================] - 318s 3s/step - loss: 2.4111 - acc: 0.4868 - val_loss: 4.5097 - val_acc: 0.0936
Epoch 4/5
101/101 [==============================] - 314s 3s/step - loss: 2.3334 - acc: 0.5396 - val_loss: 4.6063 - val_acc: 0.1182
Epoch 5/5
101/101 [==============================] - 332s 3s/step - loss: 2.3549 - acc: 0.5413 - val_loss: 4.7478 - val_acc: 0.1084


In [13]:
 # Save the model
model.save("Models/pkmn_model_best4.h5")

In [14]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5
)

Epoch 1/5
101/101 [==============================] - 313s 3s/step - loss: 2.2977 - acc: 0.5198 - val_loss: 4.6632 - val_acc: 0.1281
Epoch 2/5
101/101 [==============================] - 316s 3s/step - loss: 2.2158 - acc: 0.5726 - val_loss: 4.7062 - val_acc: 0.1034
Epoch 3/5
101/101 [==============================] - 311s 3s/step - loss: 2.2460 - acc: 0.5677 - val_loss: 4.7733 - val_acc: 0.0985
Epoch 4/5
101/101 [==============================] - 320s 3s/step - loss: 2.1336 - acc: 0.5776 - val_loss: 4.8633 - val_acc: 0.1626
Epoch 5/5
101/101 [==============================] - 312s 3s/step - loss: 2.1566 - acc: 0.5825 - val_loss: 4.6130 - val_acc: 0.1281


In [15]:
 # Save the model
model.save("Models/pkmn_model_best5.h5")